In [1]:
import re
import pandas as pd
from dwave.system import LeapHybridCQMSampler
from dimod.lp import load as loadLP

In [8]:
import dimod

In [2]:
df = pd.read_csv(r"data\knapsack\very_small.csv", names=['costs', 'weights'], header=None)

In [3]:
costs, weights = df['costs'], df['weights']

In [4]:
with open(r"D:\Projects\ProvideQ\dwave\knapsack_gams.lp", 'rb') as f:
    cqm = loadLP(f)

In [5]:
def parse_solution(sampleset, costs, weights):
    """Translate the best sample returned from solver to shipped items.

    Args:

        sampleset (dimod.Sampleset):
            Samples returned from the solver.
        costs (array-like):
            Array of costs for the items.
        weights (array-like):
            Array of weights for the items.
    """
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

    if not len(feasible_sampleset):
        raise ValueError("No feasible solution found")

    best = feasible_sampleset.first
    print(best)

    selected_item_indices = [key for key, val in best.sample.items() if val==1.0]
    
    loc_sel_items = [int(re.findall('[0-9]', ele)[0]) for ele in selected_item_indices]
    
    selected_weights = list(weights.loc[loc_sel_items])
    selected_costs = list(costs.loc[loc_sel_items])

    print("\nFound best solution at energy {}".format(best.energy))
    print("\nSelected item numbers (0-indexed):", selected_item_indices)
    print("\nSelected item weights: {}, total = {}".format(selected_weights, sum(selected_weights)))
    print("\nSelected item costs: {}, total = {}".format(selected_costs, sum(selected_costs)))

In [6]:
sampler = LeapHybridCQMSampler()

In [7]:
print("Submitting CQM to solver {}.".format(sampler.solver.name))

sampleset = sampler.sample_cqm(cqm, label='Knapsack - gamsLP1', time_limit=5)

parse_solution(sampleset, costs, weights)

Submitting CQM to solver hybrid_constrained_quadratic_model_version1.
Sample(sample={'b1': 1.0, 'b2': 0.0}, energy=-10.0, num_occurrences=1, is_satisfied=array([ True]), is_feasible=True)

Found best solution at energy -10.0

Selected item numbers (0-indexed): ['b1']

Selected item weights: [5], total = 5

Selected item costs: [10], total = 10
